In [ ]:
cd /content/drive/MyDrive/tophd/snowman-application-tasks-ay21-22/dataset

/content/drive/MyDrive/tophd/snowman-application-tasks-ay21-22/dataset


In [ ]:
import pandas as pd
import numpy as np
train = pd.read_csv("train.csv", encoding='cp1252')
test = pd.read_csv("test.csv",encoding = 'cp1252')

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.3MB 12.8MB/s 
     |████████████████████████████████| 890kB 16.7MB/s 
     |████████████████████████████████| 1.1MB 53.7MB/s 
     |████████████████████████████████| 2.9MB 60.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=aec7c8ecd5733a472d7fb86390094116ac5616861a26c44f63c82dfbbd4cb635
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification

In [ ]:
import re
def text_preprocessing(text):
    # Remove '@name'
    text = re.sub(r'(@.*?)[\s]', ' ', text)

    # Remove trailing whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [ ]:
train['Tweets'] = train['Tweets'].apply(text_preprocessing)
test['Tweets'] = test['Tweets'].apply(text_preprocessing)

In [ ]:
#Encode Label
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['label'] = le.fit_transform(train['Label'])
test['label'] = le.transform(test['Label'])

In [ ]:
train.head()

,id,Tweets,Label,label
0,7281,The jokes and puns are flying free in this cam...,none,0
1,7282,#MKR Lets see who the producers think are goin...,none,0
2,7283,Praying Jac and Shaz do well! They're my faves...,none,0
3,7284,RT Pete Evans the Paleo Capitalist has had his...,none,0
4,7285,If Kat and Andre stay tonight I will stop watc...,none,0


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train.Tweets.values, 
                                                  train.label.values, 
                                                  test_size=0.1, 
                                                  random_state=10, 
                                                  stratify=train.label.values)

In [ ]:
#bert tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

In [ ]:
#encode text

encoded_data_train = tokenizer.batch_encode_plus(
    X_train, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    X_val, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
#inputid & attention mask
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']

In [ ]:
#label to tensor
labels_train = torch.tensor(y_train)
labels_val = torch.tensor(y_val)

In [ ]:
#create dataset
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [ ]:
#create bert fine tune model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=3,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
#create Dataloader
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 16
dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),lr=1e-5, eps=1e-8)

In [ ]:
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:
#define f1 score
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [ ]:
def evaluate(dataloader_val):
  model.eval()  
  loss_val_total = 0
  predictions, true_vals = [], []
    
  for batch in dataloader_val:
    batch = tuple(b.to(device) for b in batch)    
    inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }
    with torch.no_grad():        
      outputs = model(**inputs)    
      loss = outputs[0]
      logits = outputs[1]
      loss_val_total += loss.item()

      logits = logits.detach().cpu().numpy()
      label_ids = inputs['labels'].cpu().numpy()
      predictions.append(logits)
      true_vals.append(label_ids)
    
  loss_val_avg = loss_val_total/len(dataloader_val)   
  predictions = np.concatenate(predictions, axis=0)
  true_vals = np.concatenate(true_vals, axis=0)
            
  return loss_val_avg, predictions, true_vals

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
  model.train()  
  loss_train_total = 0
  progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
  for batch in progress_bar:
    model.zero_grad() 
    batch = tuple(b.to(device) for b in batch)
        
    inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

    outputs = model(**inputs)
        
    loss = outputs[0]
    loss_train_total += loss.item()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    optimizer.step()
    scheduler.step()
        
    progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
  torch.save(model.state_dict(), f'./finetuned_BERT_epoch_{epoch}.model')
        
  tqdm.write(f'\nEpoch {epoch}')
    
  loss_train_avg = loss_train_total/len(dataloader_train)            
  tqdm.write(f'Training loss: {loss_train_avg}')
    
  val_loss, predictions, true_vals = evaluate(dataloader_validation)
  val_f1 = f1_score_func(predictions, true_vals)
  tqdm.write(f'Validation loss: {val_loss}')
  tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 0.47898431077929743
Validation loss: 0.3853042749343095
F1 Score (Weighted): 0.8395711591166181



Epoch 2
Training loss: 0.3183360342503915
Validation loss: 0.37484801921671557
F1 Score (Weighted): 0.8602502665869177



Epoch 3
Training loss: 0.24006122829769208
Validation loss: 0.4092784421633423
F1 Score (Weighted): 0.8571164276918627



Epoch 4
Training loss: 0.17398384646860846
Validation loss: 0.45601509394193135
F1 Score (Weighted): 0.8670670754654614



Epoch 5
Training loss: 0.13869673864454177
Validation loss: 0.4943322613720357
F1 Score (Weighted): 0.8597531374298699



In [ ]:
#load saved model
model.load_state_dict(torch.load('./finetuned_BERT_epoch_5.model', map_location=torch.device('cuda')))

<All keys matched successfully>

In [ ]:
#predict on test set
X_test, y_test = test.Tweets.values , test.label.values
encoded_data_test = tokenizer.batch_encode_plus(
    X_test, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)
input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(y_test)
dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)
dataloader_test = DataLoader(dataset_test, 
                                   sampler=SequentialSampler(dataset_test), 
                                   batch_size=batch_size)

_, predictions, true_vals = evaluate(dataloader_test)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
preds_flat = np.argmax(predictions, axis=1).flatten()

In [ ]:
# Set a dict for mapping id to tag name
# 0:'none', 1:'racism', 2:'sexism
tag2idx={'none': 0,'racism': 1,'sexism':2}
# Mapping index to name
tag2name={tag2idx[key] : key for key in tag2idx.keys()}

In [ ]:
y_true = []
y_pred = []
for i in range(len(true_vals)):
  y_true.append(tag2name[true_vals[i]])
  y_pred.append(tag2name[preds_flat[i]])

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

        none       0.85      0.90      0.87      2186
      racism       0.94      0.76      0.84       387
      sexism       0.65      0.58      0.62       633

    accuracy                           0.82      3206
   macro avg       0.81      0.75      0.78      3206
weighted avg       0.82      0.82      0.82      3206

